In [1]:
!pip3 install -r ../requirements.txt

In [2]:
import time

import pandas as pd
import chromedriver_binary
from bs4 import BeautifulSoup
from selenium import webdriver

## Logging in and retrieving page source code

In [3]:
WINDOW_SIZE = "1920,1080"

chrome_options = webdriver.chrome.options.Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# specifies the path to the chromedriver.exe
driver = webdriver.Chrome(options=chrome_options)

In [4]:
URL = "https://rezultatai.vrk.lt/"

In [5]:
driver.get(f'{URL}?srcUrl=/rinkimai/1104/1/1746/rezultatai/lt/rezultataiVienm.html')
time.sleep(3)

In [6]:
code = driver.page_source

In [7]:
soup = BeautifulSoup(code, 'html.parser')

In [8]:
our_table = soup.find_all("table", class_="partydata")
# our_table = str(our_table[1]).strip()

In [9]:
number_of_smds = 2 # SMD (single-member district)
sleep_time = 1 # give some time to properly load the page

In [10]:
dfs = {}
for i, each in enumerate(our_table[1].findAll("a")):
    if i >= number_of_smds:
        break
    page = str(each)[9:88]
    driver.get(URL + page)
    time.sleep(sleep_time)
    source_code = driver.page_source
    smd = driver.find_elements_by_tag_name("h3")[1].text
    print(smd)
    dfs[smd] = pd.read_html(str(source_code).strip())[1]

Senamiesčio–Žvėryno (Nr.1) apygarda
Naujamiesčio–Naujininkų (Nr.2) apygarda


In [11]:
list(dfs.values())[1]

Kandidatas  \
                      Kandidatas   
0          Žygimantas PAVILIONIS   
1     Tomas Vytautas RASKEVIČIUS   
2              Virgilijus ALEKNA   
3               Artūras MELIANAS   
4               Dmitrij IKONIKOV   
5     Ieva KAČINSKAITĖ-URBONIENĖ   
6                 Tomas SEIKALIS   
7               Antanas VALIONIS   
8                   Rita LATVYTĖ   
9                Vida MARTIŠIENĖ   
10  Astra Genovaitė ASTRAUSKAITĖ   
11                Adas DARINSKAS   
12                  Garri SERBUL   
13                           NaN   

                                               Iškėlė Paduotų balsų skaičius  \
                                               Iškėlė             apylinkėse   
0   Tėvynės sąjunga – Lietuvos krikščionys demokratai                   4179   
1                                     Laisvės partija                   1836   
2               Lietuvos Respublikos liberalų sąjūdis                   1359   
3                            Lietuvos žaliųjų partija                   1380   
4   Lietuvos lenkų rinkimų akcija - Krikščioniškų ...                   1275   
5                                       Darbo partija                   1060   
6              Lietuvos valstiečių ir žaliųjų sąjunga                    912   
7                    Lietuvos socialdemokratų partija                    776   
8                     Partija „Laisvė ir teisingumas“                    242   
9              Lietuvos socialdemokratų darbo partija                    187   
10                       Centro partija - tautininkai                    173   
11                             Partija LIETUVA – VISŲ                    167   
12                                Krikščionių sąjunga                    108   
13                                            Iš viso                  13654   

                                                                         \
   paštu iš viso % nuo galiojančių biuletenių % nuo dalyvavusių rinkėjų   
0   1206    5385                         3231                      3108   
1    557    2393                         1436                      1381   
2    289    1648                          989                       951   
3    171    1551                          931                       895   
4    150    1425                          855                       822   
5    147    1207                          724                       697   
6    148    1060                          636                       612   
7    173     949                          569                       548   
8     53     295                          177                       170   
9     40     227                          136                       131   
10    44     217                          130                       125   
11    17     184                          110                       106   
12    17     125                           75                        72   
13  3012   16666                        10000                      9618   

                       
   Unnamed: 7_level_1  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  
5                 NaN  
6                 NaN  
7                 NaN  
8                 NaN  
9                 NaN  
10                NaN  
11                NaN  
12                NaN  
13                NaN

In [12]:
# url = 'http://www.vru.gov.ua/act_list'
# df = pd.read_html(url)[0]

# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'html.parser')
# table = soup.find('table')

# links = []
# for tr in table.findAll("tr"):
#     trs = tr.findAll("td")
#     for each in trs:
#         try:
#             link = each.find('a')['href']
#             links.append(link)
#         except:
#             pass

# df['Link'] = links

- kiek apskritai yra desiniuju/kairiuju, kas laimi?
- REMOVE BEAUTIFUL SOUP